<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computational Seismology</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Parallel implementation for the advection equation I</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Collective Communications</div>
        </div>
    </div>
</div>

Seismo-Live: http://seismo-live.org

##### Authors:
* David Vargas ([@dvargas](https://github.com/davofis))
* Heiner Igel ([@heinerigel](https://github.com/heinerigel))

This notebook covers the following aspects:

* Initial configuration of setup describing input parameters for the simulation
* Parallel Matrix Multiplication
* Domain partitioning for parallel implementation
* Global communication for matrix multiplication
* Finite differences solution of the 1D advection equation 
* Visualization of the advection field in space-time
* Animated advection field

## Basic Theory
The source-free advection equation is given by

$$
\partial_t u(x, t) = v \partial_x u(x, t)
$$

where $u$ represents the advected field and $v$ velocity, and $s$. In this case, derivatives of the advection field with respect time and space can be approximated with a difference formula

$$
\partial_t u(x,t) \ \approx \ \frac{u(x,t+dt) - p(x,t)}{dt} 
$$

After introducing the approximations into the advection equation we can formulate the solution in terms of the extrapolation formula

\begin{equation}
u_{i}^{n+1} \ = \ v_i \frac{\mathrm{d}t}{ \mathrm{d}x}\left[ u_{i+1}^{n} - u_{i}^n \right] + u_{i}^n 
\end{equation}

The initial condition, $u(x,t = 0)$, may be given by a displacement waveform at $t = 0$. This waveform is advected with velocity $v$. 

### Getting started

Before you start, make sure you have launched a new Ipython cluster with the desired number of engines. Having done that, run the Ipython cluster setup cell as well as the imports cell, this will allow you to use MPI4Py into the jupyter notebook. The next cells are dedicated to the numerical implementation itself. First, the initialization of all parameters is done, you can modify those parameters to evaluate the performance and scalability of the code. Second, some methods for matrix multiplication are defined. Third, an independent cell is dedicated to perform time extrapolation, it is here where all parallel computations take place. Finally, visualization is implemented in the last part of the notebook, you will find an animated plot of the advection field as well as an image of the field itself in space-time 

### Ipython cluster setup

In [ ]:
# Import all necessary libraries, this is a configuration step for the exercise.
# Please run it before the simulation code!
from ipyparallel import Client
cluster = Client(profile='mpi')
cluster.block = True  # use synchronous computations
dview = cluster[:]
dview.activate()      # enable magics

cluster.ids

### Imports
Libraries are imported on all workers. This is a configuration step for the exercise. Please run it before the simulation code!

In [ ]:
%%px 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpi4py import MPI

import warnings
warnings.filterwarnings("ignore")
#==================================
comm = MPI.COMM_WORLD

size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()
#==================================

### 1. Method for Parallel Matrix Product
 
<div style="text-align: justify">  

<p style="width:40%;float:right;padding-left:60px;padding-right:50px">
<img src=images/Matrix_product.png>
<span style="font-size:smaller">
</span>
</p>

Matrix multiplication is a central operation in many numerical algorithms, developing efficient algorithms to perform this task is a key point in numerical analysis. Given the matrix-vector product
<br>
<br>

\begin{equation}
c_i =  \sum_{j=1}^{N} A_{ij}x_{j}
\end{equation}

<br>
We can use the concept of Toeplitz matrice, an elegant way to perform space derivatives via matrix-vector product. The strategy we follow in this notebook relies on the block partitioning approach, we split the matrix A into blocks or submatrices and assign local product as tasks per processor (see figure on the left). After all local results are available, they need to be gather into the actual global result. This is done under the global communication approach.   
</div>

In [ ]:
%%px
# Define matrix-vector parallel multiplication function
def mat_vec(comm, A, x):
    size = comm.Get_size()
    n_proc = int(A.shape[0]/size)
    if (A.shape[1] != x.size):
        print('vector-matrix size mismatch')
        comm.Abort()  
    if(x.size % size != 0):
        print('the number of processors must evenly divide n.')
        comm.Abort()
    # Initialize local matrices
    A_proc = np.zeros((n_proc, A.shape[1]))  
    # Matrix partitioning, global communicator
    comm.Scatter( [A, MPI.DOUBLE], [A_proc, MPI.DOUBLE] )
    # Broadcast vector x using global communicator
    comm.Bcast( [x, MPI.DOUBLE] )
    #Return local computation of dot product
    y_proc = A_proc @ x
    return y_proc

### 2. Initialization of setup
This cell initializes all necessary variables, arrays, and sets the initial conditions.

In [ ]:
%%px
# Parameter Configuration
#---------------------------------------------------------------
nx    = 200        # Number of grid points.
v     = 330        # Acoustic velocity in m/s.
x_max = 300        # Length of the domain in m.
eps   = 0.5        # CFL
tmax  = 1.0        # Simulation time in s
isnap = 2          # Plot the solution each `isnap` timesteps.
sig   = 10         # Sigma for the gaussian source time function
x0    = 25         # Center point of the source time function

# Initialize space
#---------------------------------------------------------------
x, dx = np.linspace(0, x_max, nx, retstep=True)

# Time step based on stability criterion
#---------------------------------------------------------------
dt = eps * dx / v
if rank == 0: print('dt =',dt) 
# Time steps
nt = int(tmax / dt)

# Initial condition
#---------------------------------------------------------------
sx = np.exp(-1.0 / sig ** 2.0 * (x - x0) ** 2.0);

# Initialize fields
#---------------------------------------------------------------
u    = sx               # u at time n (now)
unew = np.zeros(nx)     # u at time n+1 (present)
Du   = np.zeros(nx)     # 1st space derivative of u

#### 3.1 Method defining derivative matrix - Upwind scheme
<br>

<div style="text-align: justify"> 

<p style="width:40%;float:right;padding-left:60px;padding-right:50px">
\begin{equation}
D_{ij} = \frac{1}{dx}
 \begin{pmatrix}
  -1 &  1 &    &    & \\
     & -1 &  1 &    & \\
     &    & \ddots  &  &  \\
     &    &    & -1 &  1   \\
     &    &    &    & -1
 \end{pmatrix}
\end{equation}
<span style="font-size:smaller">
</span>
</p>

Numerical space derivatives are calculated by applying the differentiation matrix $D_{ij}$ to the function one seeks to derivate. The upwind scheme, 
<br>
<br>
\begin{equation}
\partial_x u(x,t) \ = \ \lim_{dx \to 0} \frac{u(x+dx,t) - u(x,t)}{dx} 
\end{equation}
<br>
can be casted into a matrix operator with the shape on the right hand side formula. The next cell defines a method implementing this matrix. 
</div>

In [ ]:
%%px
# Define derivative matrix
def FD_upwind(x, dx, nx):
    D = np.zeros((nx, nx))
    for i in range(1, nx):
        for j in range(1, nx):
            if i == j:
                D[i, j] = -1
            elif i == j + 1:
                D[i, j] = 1
            else:
                D[i, j] = 0
    D = D / dx
    return D

#### 3.2 Method defining derivative matrix - Centered scheme
<br>
<div style="text-align: justify"> 

<p style="width:40%;float:right;padding-left:60px;padding-right:50px">
\begin{equation}
D_{ij} = \frac{1}{2 dx}
 \begin{pmatrix}
   0 &  1 &    &    & \\
  -1 &  0 &  1 &    & \\
     &    & \ddots  &  &  \\
     &    & -1 &  0 &  1   \\
     &    &    & -1 &  0
 \end{pmatrix}
\end{equation}
<span style="font-size:smaller">
</span>
</p>

Similar to the upwind numerical scheme, we can formulate a centered scheme as follow. 
<br>
<br>
\begin{equation}
\partial_x u(x,t) \ = \ \lim_{dx \to 0} \frac{u(x+dx,t) - u(x-dx,t)}{2dx} 
\end{equation}
<br>
This operator is casted into a matrix form as illustrated on the right hand side formula. The next cell defines a method implementing this matrix.
</div>

In [ ]:
%%px
# Define derivative matrix
def FD_centered(x, dx, nx):
    D = np.zeros((nx, nx))
    for i in range(1, nx):
        for j in range(1, nx):
            if i == j:
                D[i, j] = 0
            elif i == j + 1:
                D[i, j] = 1
            elif i == j - 1:
                D[i, j] = -1
            else:
                D[i, j] = 0
    D = D / (2.0 * dx)
    return D

### 4. Finite Differences solution 
</p>
<div style="text-align: justify"> 

<p style="width:30%;float:right;padding-left:50px;padding-right:50px">
<img src=images/fd_toeplitz.png>
<span style="font-size:smaller">
</span>
</p>

We implement a finite difference solution using an explicit time extrapolation scheme. Since data dependency in time is involve; i.e. the future state depends on the present, this part can no be easily parallelized. 
<br>
<br>
\begin{equation}
u_{i}^{n+1} \ = \ v_i \frac{\mathrm{d}t}{ \mathrm{d}x}\left[ u_{i+1}^{n} - u_{i}^n \right] + u_{i}^n 
\end{equation}
<br>
However space derivatives can be parallelized, for instance if we consider matrix notation to perform this task
<br>
<br>

\begin{equation}
\mathbf{u}(t + dt) = v dt  \mathbf{D}\mathbf{u}(t) + \mathbf{u}(t).
\end{equation}
</div>

<br>
First we split the derivative matrix into block, then matrix multiplication occur locally on each processor, and finally we assemble the general solution using global communication. The desired shape for a derivative matrix using center differences can be observed on the right hand side.
The following cell the actual finite difference solution, please note how the gather operation is collection all local space derivatives results into a global vector 'Du'

In [ ]:
%%px
#---------------------------------------------------------------
# TIME EXTRAPOLATION 
#---------------------------------------------------------------
D = FD_upwind(x, dx, nx) # Call derivative matrix
u_xt = []                # Define array to store wavefield in space-time p(x,t)

comm.Barrier()           # Synchronize processors         
t_start = MPI.Wtime()    # start MPI timer

for it in range(nt):
    # 2nd space derivative of p. 
    Du_local = mat_vec(comm, D, u)  # local vectors defined on each processor 
    
    # Gather local vectors from workers to master, global communicator
    comm.Gather( [Du_local, MPI.DOUBLE], [Du, MPI.DOUBLE], root=0 )
    
    # --------------------------------------------------------
    # Time Extrapolation
    # --------------------------------------------------------
    unew = dt * v * Du + u
    # The new presence is the current future!
    u = unew
    # Store solution in space-time 
    u_xt.append(u)
    
comm.Barrier()    # Synchronize processors
t_final = (MPI.Wtime() - t_start)  # stop MPI timer

# Master prints time 
if (rank == 0): print('Computation time in seconds:   %s '% t_final)  
u_xt = np.asanyarray(u_xt)

### 5. Displaying the Advection Field

In [ ]:
%%px
#---------------------------------------------------------------
# Master plots advection numerical solution
#---------------------------------------------------------------
if rank == 0:
    plt.figure(figsize=(10,4))
    plt.imshow(u_xt, cmap='coolwarm', aspect='auto', extent =[0, nx*dx, nt*dt, 0])
    plt.title('u(x,t) Advection field')
    plt.ylabel('Time [s]')
    plt.xlabel('Space [m]')
    plt.show()
    #print('saving image ...')
    #plt.savefig('u_xt_I.png')    # Save your figure

### 6. Animated wave-field

In [ ]:
%%px
#---------------------------------------------------------------
# Master animates wavefield solution
#---------------------------------------------------------------
if (rank == 0):
    fig = plt.figure(figsize=(10,4))
    ax = plt.axes(xlim=(0,nx*dx), ylim=(0,1))
    line, = ax.plot([], [], lw=2, label='FDM')
    plt.title('1D Advection', fontsize=16)
    plt.ylabel('Amplitude', fontsize=12)
    plt.xlabel('x (m)', fontsize=12)
    plt.legend()
    
    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return line,
    
    # animation function. This is called sequentially
    def animate(it):
        line.set_data(x, u_xt[it, :])
        return line,
    
    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=nt, interval=40, blit=True)
    #anim.save('u_xt_I.mp4', fps=30, extra_args=['-vcodec', 'libx264']) 
    plt.show()